In [15]:
from flask import Flask, render_template, request
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from keras.utils import to_categorical
from scipy import stats 
from sklearn.preprocessing import StandardScaler

In [17]:
data = pd.read_csv('cpdata - cpdata.csv')
data

temperature   humidity        ph    rainfall       label  season  \
0       20.879744  82.002744  6.502985  202.935536        rice  Kharif   
1       21.770462  80.319644  7.038096  226.655537        rice  Kharif   
2       23.004459  82.320763  7.840207  263.964248        rice  Kharif   
3       26.491096  80.158363  6.980401  242.864034        rice  Kharif   
4       20.130175  81.604873  7.628473  262.717340        rice  Kharif   
...           ...        ...       ...         ...         ...     ...   
3095    25.287846  89.636679  6.765095   58.286977  watermelon    Zaid   
3096    26.638386  84.695469  6.189214   48.324286  watermelon    Zaid   
3097    25.331045  84.305338  6.904242   41.532187  watermelon    Zaid   
3098    26.897502  83.892415  6.463271   43.971937  watermelon    Zaid   
3099    26.986037  89.413849  6.260839   58.548767  watermelon    Zaid   

      soil type  
0     alluvial   
1     alluvial   
2     alluvial   
3     alluvial   
4     alluvial   
...         ...  
3095      sandy  
3096      sandy  
3097      sandy  
3098      sandy  
3099      sandy  

[3100 rows x 7 columns]

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3100 entries, 0 to 3099
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   temperature  3100 non-null   float64
 1   humidity     3100 non-null   float64
 2   ph           3100 non-null   float64
 3   rainfall     3100 non-null   float64
 4   label        3100 non-null   object 
 5   season       3100 non-null   object 
 6   soil type    3100 non-null   object 
dtypes: float64(4), object(3)
memory usage: 169.7+ KB


In [11]:
cols = ['temperature', 'humidity', 'ph', 'rainfall']
z_thresh = 3
def remove_outliers(df, cols):
    for col in cols:
        z_scores = np.abs((df[col] - df[col].mean()) / df[col].std())
        df = df[z_scores < 3]
    return df


In [18]:
# # Identify and remove outliers using z-score method
# data = remove_outliers(data,cols)

# Encode the label column
label_encoder = LabelEncoder()
data['label'] = label_encoder.fit_transform(data['label'])

# One-hot encode the categorical columns
categorical_cols = ['season', 'soil type']
one_hot_encoder = OneHotEncoder(sparse=False)
one_hot_encoded = one_hot_encoder.fit_transform(data[categorical_cols])
one_hot_encoded_cols = [f'{col}_{val}' for col, vals in zip(categorical_cols, one_hot_encoder.categories_) for val in vals]
data = pd.concat([data.drop(categorical_cols, axis=1), pd.DataFrame(one_hot_encoded, columns=one_hot_encoded_cols)], axis=1)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data.drop('label', axis=1), data['label'], test_size=0.2, random_state=42)

# Scale the numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the neural network model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(len(label_encoder.classes_), activation='softmax')
])

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train_scaled, y_train, epochs=50, validation_data=(X_test_scaled, y_test))

Epoch 1/50
78/78 [==============================] - 2s 9ms/step - loss: 2.4144 - accuracy: 0.4423 - val_loss: 1.1082 - val_accuracy: 0.6968
Epoch 2/50
78/78 [==============================] - 0s 4ms/step - loss: 0.5438 - accuracy: 0.8431 - val_loss: 0.3001 - val_accuracy: 0.9032
Epoch 3/50
78/78 [==============================] - 0s 4ms/step - loss: 0.2314 - accuracy: 0.9359 - val_loss: 0.1835 - val_accuracy: 0.9419
Epoch 4/50
78/78 [==============================] - 0s 4ms/step - loss: 0.1558 - accuracy: 0.9516 - val_loss: 0.1260 - val_accuracy: 0.9694
Epoch 5/50
78/78 [==============================] - 0s 4ms/step - loss: 0.1182 - accuracy: 0.9585 - val_loss: 0.1082 - val_accuracy: 0.9581
Epoch 6/50
78/78 [==============================] - 0s 4ms/step - loss: 0.0929 - accuracy: 0.9710 - val_loss: 0.0987 - val_accuracy: 0.9597
Epoch 7/50
78/78 [==============================] - 0s 4ms/step - loss: 0.0853 - accuracy: 0.9694 - val_loss: 0.0830 - val_accuracy: 0.9726
Epoch 8/50
78/78 [==

In [24]:
X_train, X_test, y_train, y_test = load_data()
to_categorical(y_train).shape

(1624, 22)

In [27]:
X_train

N   P   K  temperature   humidity        ph    rainfall
1852    7  21  35    25.760117  94.658306  5.764812  131.245141
934     8  25  36    19.913305  94.950314  6.828522  104.027706
789    60  59  22    31.868473  66.742175  7.191523   74.222386
586    15  54  15    29.976043  57.031844  8.354958   44.860529
2103  108  24  31    24.128325  56.181077  6.431900  147.275782
...   ...  ..  ..          ...        ...       ...         ...
1176   14  18  35    31.091542  47.020584  4.791147   91.466643
1397   96  18  50    25.331045  84.305338  6.904242   41.532187
906    21  21  38    22.552606  89.325949  6.327674  104.895564
1605   26  27  10    28.069032  92.914873  6.079998  114.133942
1172   20  30  27    27.810056  51.594455  4.749104   95.898986

[1624 rows x 7 columns]

In [25]:
# Define a function to build the model
def build_model():
    # Load the data
    X_train, X_test, y_train, y_test = load_data()
    y_train = to_categorical(y_train, num_classes=22)
    y_test = to_categorical(y_test, num_classes=22)
    
    # Define the model architecture
    model = Sequential()
    model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(22, activation='softmax'))
    # Compile the model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    # Train the model
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=64)
    
    return model

# Train the model
model = build_model()






Epoch 1/100
26/26 [==============================] - 1s 8ms/step - loss: 14.3475 - accuracy: 0.1034 - val_loss: 2.6786 - val_accuracy: 0.2833
Epoch 2/100
26/26 [==============================] - 0s 3ms/step - loss: 4.8092 - accuracy: 0.2180 - val_loss: 1.7437 - val_accuracy: 0.4901
Epoch 3/100
26/26 [==============================] - 0s 4ms/step - loss: 2.8379 - accuracy: 0.2734 - val_loss: 1.7945 - val_accuracy: 0.5542
Epoch 4/100
26/26 [==============================] - 0s 3ms/step - loss: 2.3830 - accuracy: 0.3220 - val_loss: 1.6549 - val_accuracy: 0.5813
Epoch 5/100
26/26 [==============================] - 0s 3ms/step - loss: 2.1820 - accuracy: 0.3830 - val_loss: 1.4642 - val_accuracy: 0.6527
Epoch 6/100
26/26 [==============================] - 0s 4ms/step - loss: 1.8572 - accuracy: 0.4631 - val_loss: 1.1522 - val_accuracy: 0.7241
Epoch 7/100
26/26 [==============================] - 0s 3ms/step - loss: 1.6737 - accuracy: 0.5135 - val_loss: 0.8884 - val_accuracy: 0.7980
Epoch 8/100


In [26]:
model.save('my_model.h5')


In [28]:
X_train

temperature   humidity        ph    rainfall  label    0    1    2    3  \
1665    26.859113  41.814208  5.131779   44.138271    9.0  0.0  1.0  0.0  0.0   
1855    30.118730  60.116808  6.578715   71.729804    0.0  0.0  1.0  0.0  0.0   
2773    11.033679  92.227068  6.562595  112.771592   24.0  0.0  0.0  0.0  1.0   
179     27.626265  52.230174  6.422694  100.316862   29.0  0.0  0.0  1.0  0.0   
1520    20.653758  23.105386  5.967533   67.717689    7.0  0.0  0.0  1.0  0.0   
...           ...        ...       ...         ...    ...  ...  ...  ...  ...   
1709    28.385039  99.188437  5.557712  189.671135    2.0  0.0  1.0  0.0  0.0   
1097    30.669383  64.330534  6.232370   78.687083    5.0  0.0  1.0  0.0  0.0   
1132    18.917127  13.527436  6.563294   45.288671   11.0  0.0  0.0  1.0  0.0   
1338    28.162393  79.645396  6.915006  141.421643   14.0  0.0  1.0  0.0  0.0   
862     26.350342  58.506502  7.460175  121.558630    3.0  0.0  1.0  0.0  0.0   

        4    5    6    7    8    9   10   11   12  
1665  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
1855  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  
2773  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  
179   0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  
1520  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  
...   ...  ...  ...  ...  ...  ...  ...  ...  ...  
1709  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
1097  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
1132  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  
1338  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  
862   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  

[2201 rows x 18 columns]

In [16]:
cols = ['temperature', 'humidity', 'ph', 'rainfall']
z_thresh = 3
def remove_outliers(df, cols, z_thresh):
    for col in cols:
        z_scores = np.abs((df[col] - df[col].mean()) / df[col].std())
        df = df[z_scores < z_thresh]
    return df

In [27]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import joblib

# Load the dataset
data = pd.read_csv('cpdata - cpdata.csv')

# Identify and remove outliers using z-score method
data = remove_outliers(data,cols,z_thresh)

# Encode the label column
label_encoder = LabelEncoder()
data['label'] = label_encoder.fit_transform(data['label'])

# One-hot encode the categorical features
cat_features = ['season', 'soil type']
one_hot_encoder = OneHotEncoder(sparse=False)
one_hot_encoded = one_hot_encoder.fit_transform(data[cat_features])
data = pd.concat([data.drop(cat_features, axis=1), pd.DataFrame(one_hot_encoded)], axis=1)

data = data.dropna()
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, :-1], data['label'], test_size=0.2, random_state=42)

# Train an ANN model
model = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=200, activation='relu', solver='adam', random_state=42)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(accuracy * 100))

# Save the encoders and trained model
joblib.dump(label_encoder, 'label_encoder.pkl')
joblib.dump(one_hot_encoder, 'one_hot_encoder.pkl')
joblib.dump(model, 'trained_model.pkl')

# Load the encoders and trained model
label_encoder = joblib.load('label_encoder.pkl')
one_hot_encoder = joblib.load('one_hot_encoder.pkl')
model = joblib.load('trained_model.pkl')





c:\Users\WARRIOR\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


Accuracy: 97.82%


c:\Users\WARRIOR\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


In [43]:
X=[20.879744,82.002744,6.502985,202.935536,'Kharif','alluvial']

In [53]:
data_df = pd.DataFrame([X],columns = ['temperature', 'humidity', 'ph', 'rainfall','season','soil type'])

In [54]:
# Encode the categorical features using one-hot encoding
cat_feature_values = data_df[cat_features].values.reshape(1, -1)
one_hot_encoded = one_hot_encoder.transform(cat_feature_values)
X_encoded = np.concatenate([data_df[['temperature', 'humidity', 'ph', 'rainfall']].values.flatten(), one_hot_encoded.flatten()])
X_encoded = X_encoded.reshape(1, -1)

c:\Users\WARRIOR\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [58]:
y = model.predict(X_encoded)

In [63]:
y[0]

2.0

In [64]:

y_pred_original = label_encoder.inverse_transform(y.astype(int))

# Print the original label
print(y_pred_original)


['Coconut']
